In [1]:
# import libaries

import numpy as np
# import scipy.sparse as sps
from sklearn.decomposition import NMF
import pandas as pd

In [2]:
# load data

ratings = pd.read_csv(r'../data/ratings.csv')
users = pd.read_csv(r'../data/users.csv', index_col=0)
movies = pd.read_csv(r'../data/movies_utf-8.csv', index_col=0)



In [3]:
# construct matrix 
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 6040 | Number of movies = 3706


In [4]:
# display matrix - note that there are missing movie_ids within the columsn

Ratings = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
Ratings

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6037,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
#normalize data to encounter different user rating habits
R = Ratings.to_numpy()
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R / 5

In [6]:
# best performing set of parameters found within hyperparemter tuning

model = NMF(n_components=50, init='random', random_state=0, verbose=True, max_iter = 10)

In [7]:
# fit model and construct feature matrix 
W = model.fit_transform(Ratings_demeaned)

H = model.components_
print("")
print("")
print("This is the user-feature-array: {}".format(W.shape))
print("This is the movie-feature-array: {}".format(H.shape))

violation: 1.0
violation: 0.4614467561507551
violation: 0.23947120097513913
violation: 0.15569804007882307
violation: 0.1112307164149588
violation: 0.08930634247271776
violation: 0.0772518064064478
violation: 0.06769385229843336
violation: 0.05969060916004253
violation: 0.05307966571940318


This is the user-feature-array: (6040, 50)
This is the movie-feature-array: (50, 3706)


In [8]:
# to do the prediction for unreviewed movies, we calculate the dot product of W and H
predictions = np.dot(W, H)

In [9]:
# precondition 1 - predicitons of baed on dot-product of NMC-Model features
# precondition 2 - original feature matrix (original predicitons)


def recommend_for_existing_user(user_id, number_of_recommendations):
  #***  PARAMS: user_id of exisiting user; number_of_recommendations for movies
    
    # get prediction for specific user
    predicted_ratings = predictions[user_id - 1]
    
    # actual rating of specific user
    rated_movies = Ratings.iloc[user_id - 1]
    
    # if not rated initially, filter is set to True
    filter_for_unreviewed_movies = [rated_movies ==0][0]

    # exhaustive filter to keep index and filter prediction for already reviewed movies
    counter = 0
    pred_ratings_unwatched_movies = []

    for state in filter_for_unreviewed_movies:        
        if state:
            # if not rated, prediction is accepted
            pred_ratings_unwatched_movies.append(predicted_ratings[counter])
        else:
            # if rated, 'prediction' is set to -1 to avoid retrieving information
            pred_ratings_unwatched_movies.append(-1)
    
        counter = counter + 1
    # sort indicies (movies) based on the value of the numpy array
    # take the desired number of recommended movies
    # flip short list around to have the indicies with the highest value on top
    
    
    # see which one works better
    #recommeded_movie_ids = np.flip(np.argsort(pred_ratings_unwatched_movies))[:number_of_recommendations]
    
    recommeded_movie_ids = np.flip((np.argsort(pred_ratings_unwatched_movies))[-number_of_recommendations:])
    #list(reversed((np.argsort(pred_ratings_unwatched_movies))[-number_of_recommendations:]))
    
    
    # we have to get the right id and add one for the actual movie-id 
    # recommendations = [x+1 for x in recommeded_movie_ids]
    
    watched_movies = [rated_movies >0][0]
    
      # exhaustive filter to keep index and filter prediction for already reviewed movies
    counter = 0
    watched_movies_only = []

    for state in watched_movies:        
        if state:
            # if not rated, prediction is accepted
            watched_movies_only.append(rated_movies.iloc[counter])
        else:
            # if rated, 'prediction' is set to -1 to avoid retrieving information
            watched_movies_only.append(-1)
    
        counter = counter + 1
    
    
    watch_list_top10_biased = np.flip((np.argsort(watched_movies_only)[-10:]))
    
    # watch_list_top10 = [x+1 for x in watch_list_top10_biased]

    watch_list_top10_biased
    recommeded_movie_ids
    # to get the actual movies
    correct_movie_ids = Ratings.columns
    
    recommendations = []
    for item in recommeded_movie_ids:
        recommendations.append(correct_movie_ids[item])
        
        
    watch_list_top10 = []
    for item in watch_list_top10_biased:
        watch_list_top10.append(correct_movie_ids[item])
    
    return watch_list_top10, recommendations

In [10]:
# user 1 example


# this user is most likely a child
most_liked_watched_movies, recommendations = recommend_for_existing_user(1, 5)

In [11]:
watched_series = pd.DataFrame()
watched_series["movie_id"] = pd.Series(list(map(int, most_liked_watched_movies))) 
pd.merge(watched_series,movies,on='movie_id')


,movie_id,title,genres
0,1,Toy Story (1995),"['Animation', ""Children's"", 'Comedy']"
1,1836,"Last Days of Disco, The (1998)",['Drama']
2,1022,Cinderella (1950),"['Animation', ""Children's"", 'Musical']"
3,1193,One Flew Over the Cuckoo's Nest (1975),['Drama']
4,48,Pocahontas (1995),"['Animation', ""Children's"", 'Musical', 'Romance']"
5,1028,Mary Poppins (1964),"[""Children's"", 'Comedy', 'Musical']"
6,1029,Dumbo (1941),"['Animation', ""Children's"", 'Musical']"
7,2804,"Christmas Story, A (1983)","['Comedy', 'Drama']"
8,1270,Back to the Future (1985),"['Comedy', 'Sci-Fi']"
9,150,Apollo 13 (1995),['Drama']


In [12]:
recommendations_series = pd.DataFrame()
recommendations_series["movie_id"] = pd.Series(list(map(int, recommendations))) 
pd.merge(recommendations_series,movies,on='movie_id')


,movie_id,title,genres
0,364,"Lion King, The (1994)","['Animation', ""Children's"", 'Musical']"
1,2081,"Little Mermaid, The (1989)","['Animation', ""Children's"", 'Comedy', 'Musical..."
2,34,Babe (1995),"[""Children's"", 'Comedy', 'Drama']"
3,318,"Shawshank Redemption, The (1994)",['Drama']
4,2080,Lady and the Tramp (1955),"['Animation', ""Children's"", 'Comedy', 'Musical..."


In [31]:
# user 738 example - more into actions and drama


# this user is most likely a child
most_liked_watched_movies, recommendations = recommend_for_existing_user(430, 5)

In [28]:
most_liked_watched_movies

[955, 3363, 905, 597, 1269, 2396, 1895, 2108, 858, 920]

In [29]:
recommendations

[357, 11, 1307, 1393, 2424]

In [32]:
watched_series = pd.DataFrame()
watched_series["movie_id"] = pd.Series(list(map(int, most_liked_watched_movies))) 
pd.merge(watched_series,movies,on='movie_id')


,movie_id,title,genres
0,955,Bringing Up Baby (1938),['Comedy']
1,3363,American Graffiti (1973),"['Comedy', 'Drama']"
2,905,It Happened One Night (1934),['Comedy']
3,597,Pretty Woman (1990),"['Comedy', 'Romance']"
4,1269,Arsenic and Old Lace (1944),"['Comedy', 'Mystery', 'Thriller']"
5,2396,Shakespeare in Love (1998),"['Comedy', 'Romance']"
6,1895,Can't Hardly Wait (1998),"['Comedy', 'Drama', 'Romance']"
7,2108,L.A. Story (1991),"['Comedy', 'Romance']"
8,858,"Godfather, The (1972)","['Action', 'Crime', 'Drama']"
9,920,Gone with the Wind (1939),"['Drama', 'Romance', 'War']"


In [26]:
recommendations_series = pd.DataFrame()
recommendations_series["movie_id"] = pd.Series(list(map(int, recommendations))) 
pd.merge(recommendations_series,movies,on='movie_id')


,movie_id,title,genres
0,2396,Shakespeare in Love (1998),"['Comedy', 'Romance']"
1,2692,Run Lola Run (Lola rennt) (1998),"['Action', 'Crime', 'Romance']"
2,1197,"Princess Bride, The (1987)","['Action', 'Adventure', 'Comedy', 'Romance']"
3,260,Star Wars: Episode IV - A New Hope (1977),"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']"
4,1198,Raiders of the Lost Ark (1981),"['Action', 'Adventure']"


In [ ]:
#export prediction matrix and actual Rating-Matrix

In [19]:
import pickle
file_name = '../export/predictions-v2.pkl'
pickle.dump(predictions, open(file_name, 'wb'))
 
# We'll use this to reload from our Streamlit app
#predictions = pickle.load(open(file_name, 'rb'))

In [68]:
# export the ratings for the correct mapping of index to movie_id
Ratings.to_csv('../export/user-movie-matrix.csv')

In [80]:
# load Ratings and predictions
Ratings= pd.read_csv(r'../export/user-movie-matrix.csv', index_col=0)
predictions = pickle.load(open(file_name, 'rb'))

In [18]:
# precondition 1 - predicitons of baed on dot-product of NMC-Model features
# precondition 2 - original feature matrix (original predicitons)


def recommend_for_existing_user(user_id, number_of_recommendations):
  #***  PARAMS: user_id of exisiting user; number_of_recommendations for movies
    
    # get prediction for specific user
    predicted_ratings = predictions[user_id - 1]
    
    # actual rating of specific user
    rated_movies = Ratings.iloc[user_id - 1]
    
    # if not rated initially, filter is set to True
    filter_for_unreviewed_movies = [rated_movies ==0][0]

    # exhaustive filter to keep index and filter prediction for already reviewed movies
    counter = 0
    pred_ratings_unwatched_movies = []

    for state in filter_for_unreviewed_movies:        
        if state:
            # if not rated, prediction is accepted
            pred_ratings_unwatched_movies.append(predicted_ratings[counter])
        else:
            # if rated, 'prediction' is set to -1 to avoid retrieving information
            pred_ratings_unwatched_movies.append(-1)
    
        counter = counter + 1
    # sort indicies (movies) based on the value of the numpy array
    # take the desired number of recommended movies
    # flip short list around to have the indicies with the highest value on top
    
    
    # see which one works better
    #recommeded_movie_ids = np.flip(np.argsort(pred_ratings_unwatched_movies))[:number_of_recommendations]
    
    recommeded_movie_ids = np.flip((np.argsort(pred_ratings_unwatched_movies))[-number_of_recommendations:])
    #list(reversed((np.argsort(pred_ratings_unwatched_movies))[-number_of_recommendations:]))
    
    
    # we have to get the right id and add one for the actual movie-id 
    # recommendations = [x+1 for x in recommeded_movie_ids]
    
    watched_movies = [rated_movies >0][0]
    
      # exhaustive filter to keep index and filter prediction for already reviewed movies
    counter = 0
    watched_movies_only = []

    for state in watched_movies:        
        if state:
            # if not rated, prediction is accepted
            watched_movies_only.append(rated_movies.iloc[counter])
        else:
            # if rated, 'prediction' is set to -1 to avoid retrieving information
            watched_movies_only.append(-1)
    
        counter = counter + 1
    
    
    watch_list_top10_biased = np.flip((np.argsort(watched_movies_only)[-10:]))
    
    # watch_list_top10 = [x+1 for x in watch_list_top10_biased]

    watch_list_top10_biased
    recommeded_movie_ids
    # to get the actual movies
    correct_movie_ids = Ratings.columns
    
    recommendations = []
    for item in recommeded_movie_ids:
        recommendations.append(correct_movie_ids[item])
        
        
    watch_list_top10 = []
    for item in watch_list_top10_biased:
        watch_list_top10.append(correct_movie_ids[item])
    
    return watch_list_top10, recommendations

In [21]:
most_liked_watched_movies, recommendations = recommend_for_existing_user(619, 3)
print(most_liked_watched_movies)
print(recommendations)

[2324, 2858, 3083, 1220, 2248, 1230, 3911, 1244, 2804, 2599]
[2396, 2692, 1197]
